### Guardrails for Amazon Bedrock - Examples using the Python SDK


<center><img src="images/MLU-NEW-logo.png" alt="drawing" width="400" style="background-color:white; padding:1em;" /></center> <br/>

# <a name="0">Adversarial Robustness using Bedrock Agents and Bedrock Guardrails</a>

## <a name="0"> Part 1a: Create a Bedrock based Guardrail to prevent Fiduciary/Financial advice </a>

## Notebook Overview


This repository on Robustness, uses Bedrock Agents and Bedrock Guardrails to demonstrate Adversarial Robustness efficacy.

- 1a. [THIS NOTEBOOK] Create Guardrail against fiduciary advice

- 1b.  Demonstrate bedrock agents - retail bot use-case WITHOUT guardrails to show the adversarial robustness concern.

- 1c.  Demonstrate bedrock agents - retail bot use-case WITH guardrails to improve and evaluate the adversarial robustness concern.

In this part 1a,  we will create a Bedrock Guardrail with the intention to block/deny a retail-bot using Bedrock agent from giving any financial or fiduciary advice.

##### Notebook Kernel
Please choose `conda_python3` as the kernel type of the top right corner of the notebook if that does not appear by default.


<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>This notebook automatically cleans up resources to be frugal. </h4>
    You can visit this section (<a href="#9"> Clean-up Resources</a>) to change the setting if you need to experiment with prompts and settings. Please run clean-up resources after you are done with experiments. <br/>
</div>
<br/>

In this use-case, we use Bedrock Guardrails API (https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails.html) with BOTO3 to create a bedrock guardrail that prevents fiduciary/financial advice.

This notebook will demonstrate how to create your first agent for Bedrock using the Boto3 SDK:

1. Create a Amazon Bedrock Guardrail using Boto3 API
2. Listing and creating Guardrail versions 
3. Updating Guardrails
4. Unit-testing Bedrock Guardrails
5. Capture the guardrailId and guardrailArn



We provide a step-by-step guide of creating and listing bedrock guardrails using guardrails API via Boto3. 

This notebook has the following sections:

1. <a href="#1">Create a Amazon Bedrock Guardrail using BOTO3 API</a>
2. <a href="#2">Listing and creating Guardrail versions </a>
3. <a href="#3"> Updating Guardrails</a>
4. <a href="#4"> Unit-testing Bedrock Guardrails </a>
4. <a href="#5"> [Notebook Takeaway] Capture the guardrailId and guardrailArn </a>

    
Please work top to bottom of this notebook and don't skip sections as this could lead to error messages due to missing code.

---



Guardrails for Amazon Bedrock evaluates user inputs and FM responses based on use case specific policies, and provides an additional layer of safeguards regardless of the underlying FM. Guardrails can be applied across all large language models (LLMs) on Amazon Bedrock, including fine-tuned models. Customers can create multiple guardrails, each configured with a different combination of controls, and use these guardrails across different applications and use cases. 


In [1]:
%%capture
#Start by installing the dependencies to ensure we have a recent version
!pip install --upgrade --force-reinstall boto3


In [2]:
import boto3
print(boto3.__version__)

1.34.132


In [3]:
client = boto3.client('bedrock')

## <a name="1"> Create a Amazon Bedrock Guardrail using BOTO3 API </a>
(<a href="#0">Go to top</a>)


Guardrails for Amazon Bedrock have multiple components which include Content Filters, Denied Topics, Word and Phrase Filters, and Sensitive Word (PII & Regex) Filters. For a full list check out the [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-create.html) 

In [4]:
# We'll create a guardrail to prevent providing Fiduciary Advice

create_response = client.create_guardrail(
    name='fiduciary-advice-v2',
    description='Prevents the our model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice v2',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
   contentPolicyConfig={
    'filtersConfig': [
        {
            'type': 'SEXUAL',
            'inputStrength': 'HIGH',
            'outputStrength': 'HIGH'
        },
        {
            'type': 'VIOLENCE',
            'inputStrength': 'HIGH',
            'outputStrength': 'HIGH'
        },
        {
            'type': 'HATE',
            'inputStrength': 'HIGH',
            'outputStrength': 'HIGH'
        },
        {
            'type': 'INSULTS',
            'inputStrength': 'HIGH',
            'outputStrength': 'HIGH'
        },
        {
            'type': 'MISCONDUCT',
            'inputStrength': 'HIGH',
            'outputStrength': 'HIGH'
        },
        {
            'type': 'PROMPT_ATTACK',
            'inputStrength': 'HIGH',
            'outputStrength': 'NONE'
        }
    ]

    },
wordPolicyConfig={
    'wordsConfig': [
        {
            'text': 'fiduciary advice'
        },
        {
            'text': 'investment recommendations'
        },
        {
            'text': 'stock picks'
        },
        {
            'text': 'financial planning guidance'
        },
        {
            'text': 'portfolio allocation advice'
        },
        {
            'text': 'retirement fund suggestions'
        },
        {
            'text': 'wealth management tips'
        },
        {
            'text': 'trust fund setup'
        },
        {
            'text': 'investment strategy'
        },
        {
            'text': 'financial advisor recommendations'
        }
    ],
    'managedWordListsConfig': [
        {
            'type': 'PROFANITY'
        }
    ]
},
sensitiveInformationPolicyConfig={
    'piiEntitiesConfig': [
        {
            'type': 'EMAIL',
            'action': 'ANONYMIZE'
        },
        {
            'type': 'PHONE',
            'action': 'ANONYMIZE'
        },
        {
            'type': 'NAME',
            'action': 'ANONYMIZE'
        },
        {
            'type': 'US_SOCIAL_SECURITY_NUMBER',
            'action': 'BLOCK'
        },
        {
            'type': 'US_BANK_ACCOUNT_NUMBER',
            'action': 'BLOCK'
        },
        {
            'type': 'CREDIT_DEBIT_CARD_NUMBER',
            'action': 'BLOCK'
        }
    ],
    'regexesConfig': [
        {
            'name': 'Account Number',
            'description': 'Matches account numbers in the format XXXXXX1234',
            'pattern': r'\b\d{6}\d{4}\b',
            'action': 'ANONYMIZE'
        }
    ]
},
    blockedInputMessaging='I apologize, but I am not able to provide fiduciary advice. Additionally, it seems that you may have included some sensitive personal or financial information in your request. For your privacy and security, please modify your input and try again without including any personal, financial, or restricted details.',
    blockedOutputsMessaging='I apologize, but I am not able to provide fiduciary advice. Additionally, it seems that you may have included some sensitive personal or financial information in your request. For your privacy and security, please modify your input and try again without including any personal, financial, or restricted details.',
)

print(create_response)

{'ResponseMetadata': {'RequestId': 'd49d3700-0ea5-4de9-95f0-2897d7f3eb3f', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Tue, 25 Jun 2024 01:03:45 GMT', 'content-type': 'application/json', 'content-length': '172', 'connection': 'keep-alive', 'x-amzn-requestid': 'd49d3700-0ea5-4de9-95f0-2897d7f3eb3f'}, 'RetryAttempts': 0}, 'guardrailId': 'npv8r9lgjo92', 'guardrailArn': 'arn:aws:bedrock:us-east-1:757420736997:guardrail/npv8r9lgjo92', 'version': 'DRAFT', 'createdAt': datetime.datetime(2024, 6, 25, 1, 3, 45, 379844, tzinfo=tzlocal())}


## <a name="2"> Listing and creating Guardrail versions </a>
(<a href="#0">Go to top</a>)

In [5]:
#This will provide all the data about the DRAFT version we have
get_response = client.get_guardrail(
    guardrailIdentifier=create_response['guardrailId'],
    guardrailVersion='DRAFT'
)


In [6]:
# Now let's create a version for our Guardrail 
version_response = client.create_guardrail_version(
    guardrailIdentifier=create_response['guardrailId'],
    description='Version of Guardrail that has HIGH content filters across'
)

In [7]:
# To list the DRAFT version of all your guardrails, don’t specify the guardrailIdentifier field. To list all versions of a guardrail, specify the ARN of the guardrail in the guardrailIdentifier field.
list_guardrails_response = client.list_guardrails(
    guardrailIdentifier=create_response['guardrailArn'],
    maxResults=5)

print(list_guardrails_response)

{'ResponseMetadata': {'RequestId': '6bf80c86-f5a1-4511-9521-1f9102206f7d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 25 Jun 2024 01:03:46 GMT', 'content-type': 'application/json', 'content-length': '675', 'connection': 'keep-alive', 'x-amzn-requestid': '6bf80c86-f5a1-4511-9521-1f9102206f7d'}, 'RetryAttempts': 0}, 'guardrails': [{'id': 'npv8r9lgjo92', 'arn': 'arn:aws:bedrock:us-east-1:757420736997:guardrail/npv8r9lgjo92', 'status': 'VERSIONING', 'name': 'fiduciary-advice-v2', 'description': 'Prevents the our model from providing fiduciary advice.', 'version': 'DRAFT', 'createdAt': datetime.datetime(2024, 6, 25, 1, 3, 45, 379844, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2024, 6, 25, 1, 3, 45, 486525, tzinfo=tzlocal())}, {'id': 'npv8r9lgjo92', 'arn': 'arn:aws:bedrock:us-east-1:757420736997:guardrail/npv8r9lgjo92', 'status': 'CREATING', 'name': 'fiduciary-advice-v2', 'description': 'Version of Guardrail that has HIGH content filters across', 'version': '1', 'createdAt':

## <a name="3"> Updating Guardrails and list versions </a>
(<a href="#0">Go to top</a>)

Let's update the Guardrail but this time modify one of our content filters.

In [10]:
# Updating the Guardrail by providing another adjusting our Content Filter strength 

response = client.update_guardrail(
    guardrailIdentifier=create_response['guardrailArn'],
    name='fiduciary-advice-v2',
    description='Prevents the our model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Avoid providing guidance on managing financial assets, investments, or trusts to prevent fiduciary responsibility.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
   contentPolicyConfig={
    'filtersConfig': [
        {
            'type': 'SEXUAL',
            'inputStrength': 'HIGH',
            'outputStrength': 'HIGH'
        },
        {
            'type': 'VIOLENCE',
            'inputStrength': 'HIGH',
            'outputStrength': 'HIGH'
        },
        {
            'type': 'HATE',
            'inputStrength': 'HIGH',
            'outputStrength': 'MEDIUM' #field that was edited
        },
        {
            'type': 'INSULTS',
            'inputStrength': 'HIGH',
            'outputStrength': 'HIGH'
        },
        {
            'type': 'MISCONDUCT',
            'inputStrength': 'HIGH',
            'outputStrength': 'HIGH'
        },
        {
            'type': 'PROMPT_ATTACK',
            'inputStrength': 'HIGH',
            'outputStrength': 'NONE'
        }
    ]

    },
wordPolicyConfig={
    'wordsConfig': [
        {
            'text': 'fiduciary advice'
        },
        {
            'text': 'investment recommendations'
        },
        {
            'text': 'stock picks'
        },
        {
            'text': 'financial planning guidance'
        },
        {
            'text': 'portfolio allocation advice'
        },
        {
            'text': 'retirement fund suggestions'
        },
        {
            'text': 'wealth management tips'
        },
        {
            'text': 'trust fund setup'
        },
        {
            'text': 'investment strategy'
        },
        {
            'text': 'financial advisor recommendations'
        }
    ],
    'managedWordListsConfig': [
        {
            'type': 'PROFANITY'
        }
    ]
},
sensitiveInformationPolicyConfig={
    'piiEntitiesConfig': [
        {
            'type': 'EMAIL',
            'action': 'ANONYMIZE'
        },
        {
            'type': 'PHONE',
            'action': 'ANONYMIZE'
        },
        {
            'type': 'NAME',
            'action': 'ANONYMIZE'
        },
        {
            'type': 'US_SOCIAL_SECURITY_NUMBER',
            'action': 'BLOCK'
        },
        {
            'type': 'US_BANK_ACCOUNT_NUMBER',
            'action': 'BLOCK'
        },
        {
            'type': 'CREDIT_DEBIT_CARD_NUMBER',
            'action': 'BLOCK'
        }
    ],
    'regexesConfig': [
        {
            'name': 'Account Number',
            'description': 'Matches account numbers in the format XXXXXX1234',
            'pattern': r'\b\d{6}\d{4}\b',
            'action': 'ANONYMIZE'
        }
    ]
},
    blockedInputMessaging='I apologize, but I am not able to provide fiduciary advice. Additionally, it seems that you may have included some sensitive personal or financial information in your request. For your privacy and security, please modify your input and try again without including any personal, financial, or restricted details.',
    blockedOutputsMessaging='I apologize, but I am not able to provide fiduciary advice. Additionally, it seems that you may have included some sensitive personal or financial information in your request. For your privacy and security, please modify your input and try again without including any personal, financial, or restricted details.',
)

In [11]:
# Let's now get all of our updates 
get_response = client.get_guardrail(
    guardrailIdentifier=create_response['guardrailId'],
    guardrailVersion='DRAFT'
)
print(get_response)

{'ResponseMetadata': {'RequestId': '9fe5f0dc-c8c3-4815-aff9-5ed2aab329ab', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 25 Jun 2024 01:04:22 GMT', 'content-type': 'application/json', 'content-length': '2904', 'connection': 'keep-alive', 'x-amzn-requestid': '9fe5f0dc-c8c3-4815-aff9-5ed2aab329ab'}, 'RetryAttempts': 0}, 'name': 'fiduciary-advice-v2', 'description': 'Prevents the our model from providing fiduciary advice.', 'guardrailId': 'npv8r9lgjo92', 'guardrailArn': 'arn:aws:bedrock:us-east-1:757420736997:guardrail/npv8r9lgjo92', 'version': 'DRAFT', 'status': 'READY', 'topicPolicy': {'topics': [{'name': 'Fiduciary Advice', 'definition': 'Avoid providing guidance on managing financial assets, investments, or trusts to prevent fiduciary responsibility.', 'examples': ['What stocks should I invest in for my retirement?', 'Is it a good idea to put my money in a mutual fund?', 'How should I allocate my 401(k) investments?', 'What type of trust fund should I set up for my children?', 

In [12]:
# Create a new Version from our updates 
version_response = client.create_guardrail_version(
    guardrailIdentifier=create_response['guardrailId'],
    description='Version of Guardrail that has a MEDIUM Hate Filter'
)

In [13]:
# Get all of our Guardrails 
list_guardrails_response = client.list_guardrails(
    guardrailIdentifier=create_response['guardrailArn'],
    maxResults=5)

In [14]:
list_guardrails_response

{'ResponseMetadata': {'RequestId': '9fc530fb-8765-4db7-9285-2a47e096eba7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 25 Jun 2024 01:04:29 GMT',
   'content-type': 'application/json',
   'content-length': '946',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9fc530fb-8765-4db7-9285-2a47e096eba7'},
  'RetryAttempts': 0},
 'guardrails': [{'id': 'npv8r9lgjo92',
   'arn': 'arn:aws:bedrock:us-east-1:757420736997:guardrail/npv8r9lgjo92',
   'status': 'READY',
   'name': 'fiduciary-advice-v2',
   'description': 'Prevents the our model from providing fiduciary advice.',
   'version': 'DRAFT',
   'createdAt': datetime.datetime(2024, 6, 25, 1, 3, 45, tzinfo=tzlocal()),
   'updatedAt': datetime.datetime(2024, 6, 25, 1, 4, 28, 436278, tzinfo=tzlocal())},
  {'id': 'npv8r9lgjo92',
   'arn': 'arn:aws:bedrock:us-east-1:757420736997:guardrail/npv8r9lgjo92',
   'status': 'READY',
   'name': 'fiduciary-advice-v2',
   'description': 'Version of Guardrail that has HIGH content filters

## <a name="4"> Unit-testing Bedrock Guardrails </a>
(<a href="#0">Go to top</a>)

In [15]:
# use the bedrock run-time client to invoke the guardrails
import json
bedrock_runtime = boto3.client('bedrock-runtime')

In [16]:
# Build our request to Bedrock, we will test our second version

payload = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "How should I invest for my retirement? I want to be able to generate $5,000 a month"
                    }
                ]
            }
        ]
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
response = bedrock_runtime.invoke_model(
    body = body_bytes,
    contentType = payload['contentType'],
    accept = payload['accept'],
    modelId = payload['modelId'],
    guardrailIdentifier = create_response['guardrailId'], 
    guardrailVersion ="2", 
    trace = "ENABLED"
)

# Print the response
response_body = response['body'].read().decode('utf-8')
print(response_body)


{"type":"message","role":"assistant","content":[{"type":"text","text":"I apologize, but I am not able to provide fiduciary advice. Additionally, it seems that you may have included some sensitive personal or financial information in your request. For your privacy and security, please modify your input and try again without including any personal, financial, or restricted details."}],"amazon-bedrock-trace":{"guardrail":{"input":{"npv8r9lgjo92":{"topicPolicy":{"topics":[{"name":"Fiduciary Advice","type":"DENY","action":"BLOCKED"}]}}}}},"amazon-bedrock-guardrailAction":"INTERVENED"}


## <a name="5"> Notebook Takeaway : Capture the guardrailId and guardrailArn </a>
(<a href="#0">Go to top</a>)



From the guardrail create and update API calls, we can capture the guardrail parameters (`guardrailId` and `guardrailArn`) to associate them with Bedrock Agents in the next notebook. An example parameter:value is (DO NOT USE THESE SAMPLE VALUES)

 - `guardrailId: an9l3icjg3kj` 
 - `guardrailArn: arn:aws:bedrock:us-east-1:757420736997:guardrail/an9l3icjg3kj`

In [24]:
# the guardrailId created in this notebook and to be used
create_response['guardrailId']

'npv8r9lgjo92'

In [23]:
# the guardrailArn created in this notebook and to be used
create_response['guardrailArn']

'arn:aws:bedrock:us-east-1:757420736997:guardrail/npv8r9lgjo92'